In [1]:
#Downloading all the required libraries
!pip3 install gym
!pip3 install gym-super-mario-bros
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

In [3]:
from gym.version import VERSION
print(VERSION)

0.26.2


In [2]:
import os 

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import FrameStack, GrayScaleObservation

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Using PPO Algo for reinf learning
from stable_baselines3 import PPO
#To save the model
from stable_baselines3.common.callbacks import BaseCallback

import numpy as np
from matplotlib import pyplot as plt

Pre-processing

In [65]:
#Setting the environment for the game
env = gym_super_mario_bros.make('SuperMarioBros-v1',apply_api_compatibility=True,render_mode="human" )
print('No.of button Combinations before using JoypadSpace Wrapper :'+ str(env.action_space))

env = JoypadSpace(env, SIMPLE_MOVEMENT) #Reduces the no.of combinations
print('No.of button Combinations after using JoypadSpace Wrapper :'+ str(env.action_space))

No.of button Combinations before using JoypadSpace Wrapper :Discrete(256)
No.of button Combinations after using JoypadSpace Wrapper :Discrete(7)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v1 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [66]:
#Converting the color img frame to grayscale
#Makes it easier to train the model as data size will be reduced
state=env.reset()
env= GrayScaleObservation(env, keep_dim=True)
env=DummyVecEnv([lambda:env])
env = VecFrameStack(env,4, channels_order='last')
state, reward, done, info = env.step([5])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [67]:
class Callbacks(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(Callbacks, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'model_{self.n_calls}')
            self.model.save(model_path)


        return True

In [68]:
CHECKPOINT_DIR = './model/'
LOG_DIR = './model/logs'

In [69]:
callback = Callbacks(check_freq=100000, save_path=CHECKPOINT_DIR)

In [70]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512) 

Using cpu device
Wrapping the env in a VecTransposeImage.


In [64]:
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)

In [51]:
model.learn(total_timesteps=5000000, callback=callback)

Logging to ./logs/PPO_7


----------------------------
| time/              |     |
|    fps             | 118 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------


KeyboardInterrupt: 

In [20]:
model.save('Final_model')

Testing

In [40]:
model = PPO.load('./best_model_5000000')

In [41]:
state = env.reset()

In [42]:
# Loading the game with our trained ml model
state = env.reset()
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

KeyboardInterrupt: 